In [1]:
from obspy.core import Stream, read
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl 
from matplotlib.colors import Normalize
import pandas as pd
import os 
import os.path

In [2]:
path_to_files = '/Users/sydneydybing/StrainProject/2024/'

earthquakes = pd.read_csv(path_to_files + 'M6_catalog_2004-2024.csv', dtype = str)

weird_events_only = False
if weird_events_only:
    earthquakes = earthquakes.iloc[[4,15]]

small_test = False
if small_test:
    earthquakes = earthquakes[5:7]
    
one_at_a_time = False
if one_at_a_time:
    earthquakes = earthquakes[:1]

origin_times = earthquakes.time.values
mags = earthquakes.mag.values

bsm_mda = pd.read_csv(path_to_files + 'bsm_metadata.csv')
NA_only = bsm_mda.loc[bsm_mda['LONG'] < -110]
stas = NA_only['BNUM'].values

eqs_test = earthquakes[4:5]
ots_test = origin_times[4:5]
mags_test = mags[4:5]
rows_for_extra_times = [8,9,14,16]

dist_array = np.load('/Users/sydneydybing/StrainProject/2024/NA_evt-sta_dist_array_less500km_withdata.npy')

# Array order:

# 0. Earthquake identifier
# 1. Earthquake origin time
# 2. Earthquake latitude
# 3. Earthquake longitude
# 4. Earthquake magnitude
# 5. Station name
# 6. Station latitude
# 7. Station longitude
# 8. Hypocentral distance (km)

In [3]:
dist_array[0]

array(['2023-04-13_M6', '2023-04-13T15:54:53.808Z', '49.215', '-129.6178',
       '6.0', 'B001', '48.04307', '-123.13141', '495.399747368'],
      dtype='<U32')

In [4]:
# k = np.where(dist_array[:,0] == '2023-04-13_M6')[0] # To just run one event
rms_snrs = []

for i in range(len(dist_array)):
    
    eq_id = dist_array[i,0]
    sta = dist_array[i,5]
    hypdist = float(dist_array[i,8])
    
#     print(eq_id, sta)
    
    try:
        strain_waves = read(path_to_files + 'strain_data/3comp_trimmed_rms/' + str(eq_id) + '/' + str(sta) + '.mseed')
    
    except:
        rms_snrs.append(np.nan)
        continue # skips to next row in dist_array
    
    times = strain_waves[0].times()
    samples = np.arange(strain_waves[0].stats.npts)
    data = strain_waves[0].data
    
    samp_rate = strain_waves[0].stats.sampling_rate
    
    samples_to_p_pick = 10 * samp_rate
    
        
    if hypdist < 100:
#         print('Close one: ' + str(round(hypdist)) + ' km')
#         print(eq_id, sta)
        k = np.where(data != 0)[0]
#         print('Data is nonzero at indices:')
#         print(k)
#         print('Data starts being nonzero at time:')
#         print(times[k[0]])
#         print('We have ' + str(9 - times[k[0]]) + ' seconds of useable noise data')
        
        if float(9 - times[k[0]]) >= 8:
            noise_start = int(samples_to_p_pick - 9*samp_rate)
        else:
            noise_start = k[0]
            
#         print('First noise sample is:')
#         print(noise_start)
#         print('')
    
    else:
        noise_start = int(samples_to_p_pick - 9*samp_rate) # 8 seconds of noise
    
    noise_end = int(samples_to_p_pick - 1*samp_rate)
    
    # Plot to check noise window positioning
#     if hypdist < 100: 
#         plt.plot(samples, data, color = 'black')
#         plt.semilogy()
#         plt.axvline(x = 10*samp_rate, color = 'blue')
#         plt.axvline(x = noise_start, color = 'green')
#         plt.axvline(x = noise_end, color = 'red')
#         plt.xlim(-1*samp_rate,30*samp_rate)
#         plt.show()

    signal_start = int(samples_to_p_pick)
    signal_end = int(samples_to_p_pick + 10*samp_rate) # 30 seconds of signal
    
    signal = data[signal_start:signal_end]
    noise = data[noise_start:noise_end]
    
    noise_rms = np.sqrt(np.mean(noise**2))
    signal_rms = np.sqrt(np.mean(signal**2))
    snr = signal_rms/noise_rms
#     snr_db = 20*np.log10(snr)
    
    rms_snrs.append(snr)
    
#     print(snr_db)
    
    pst = read(path_to_files + 'strain_data/3comp_peak/' + str(eq_id) + '/' + str(sta) + '.mseed')
    seis_waves = read(path_to_files + 'seismic_data/trimmed/' + str(eq_id) + '/' + str(sta) + '.mseed')
    
    # Plot all 3 waveforms
    
    fig = plt.figure(figsize = (10,6), dpi = 300, facecolor = 'white', num = 1, clear = True)
    ax = fig.add_subplot(111)
    ax.grid()
    ax.set_title('Earthquake ' + str(eq_id) + ', station ' + str(sta) + '\nRMS strain SNR: ' + str(round(snr,2)), fontsize = 14)
    ax.plot(pst[0].times(), pst[0].data*10**9, color = 'red', label = 'Peak strain')
    ax.plot(times, data*10**9, color = 'blue', label = 'RMS strain')
    ax2 = ax.twinx()
    ax2.plot(seis_waves[0].times(), seis_waves[0].data, color = 'green', alpha = 0.5, label = 'Seismic data')
    ax.set_yscale('log')
#     ax.set_ylabel('Log of peak extensional\nmicrostrain ($10^{-6}$)', fontsize = 15)
    ax.set_xlabel('Time (s)', fontsize = 15)
    ax.set_xlim(0,40)
#     ax.set_xticks([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])
    ax.axvline(x = 10, color = 'gray', linestyle = '--')
    ax.tick_params(labelsize = 13)
    ax.legend(loc = 'upper left', fontsize = 12)
    ax2.legend(loc = 'lower right', fontsize = 12)
    
#     plt.show()
    
    # Create folder for the event
    eq_sta_save_dir = path_to_files + 'figures/3comp_rms_peak_seismic_overlay_plots/' + str(eq_id) + '/'
#     print(eq_sta_save_dir)
    if os.path.isdir(eq_sta_save_dir):
        pass
    else:
        os.makedirs(eq_sta_save_dir)
    
    plt.savefig(eq_sta_save_dir + str(sta) + '.png', format = 'PNG')
    plt.close();
    
print(len(rms_snrs))

/var/folders/yg/s8qp7lfs59g_tbwsx_wxkdx80000gn/T/ipykernel_9373/3272500772.py:70: RuntimeWarning: invalid value encountered in double_scalars
  snr = signal_rms/noise_rms
/var/folders/yg/s8qp7lfs59g_tbwsx_wxkdx80000gn/T/ipykernel_9373/3272500772.py:90: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  ax.set_yscale('log')


439


In [5]:
reshape_rms = np.array(rms_snrs).reshape(len(rms_snrs),1)

In [6]:
new_array = np.hstack((dist_array, reshape_rms))

# Array order:

# 0. Earthquake identifier
# 1. Earthquake origin time
# 2. Earthquake latitude
# 3. Earthquake longitude
# 4. Earthquake magnitude
# 5. Station name
# 6. Station latitude
# 7. Station longitude
# 8. Hypocentral distance (km)
# 9. RMS strain SNR

In [7]:
new_array[0]

array(['2023-04-13_M6', '2023-04-13T15:54:53.808Z', '49.215', '-129.6178',
       '6.0', 'B001', '48.04307', '-123.13141', '495.399747368',
       '1.24916559849'],
      dtype='<U32')

In [8]:
np.save('/Users/sydneydybing/StrainProject/2024/NA_evt-sta_dist_array_less500km_withdata_withrmssnrs_3COMP.npy', new_array)

In [9]:
test = np.load('/Users/sydneydybing/StrainProject/2024/NA_evt-sta_dist_array_less500km_withdata_withrmssnrs_3COMP.npy')

In [10]:
test.shape

(439, 10)